## __Optimizers__

We already implemented something along the lines of an optimizer, in our training loop. But now, we will be creating a class for the optimizer. The kind of optimizer we are dealing with is called Gradient Descent. Gradient Descent is a form of optimization, that uses the gradients of weights and biases, to take incremental steps towards the minima of the loss function, in an effort to decrease loss. Let's create a class called Gradient Descent, to represent this. We will set default LR as 0.001

In [9]:
class Optimizer_GD:
  def __init__(self, learning_rate=0.001):
    self.learning_rate = learning_rate
  def update_params(self, layer):
    layer.weights -= self.learning_rate * layer.dweights
    layer.biases -= self.learning_rate * layer.dbiases

### __Decay and Momentum__

Let's first talk about decay. Let's say we start off with a high learning rate, so we can jump quickly towards the global minimum of the cost function. But as we get closer, we may want to decrease our learning rate, since we don't want to jump around the cost function as much. To lower the learning rate over iterations is known as decay. The formula for decay is given below.

#### $LR = \frac {LR}{(1 + R_{d} t)}$

Here $R_{d}$ represents the rate of decay, and $t$ represents number of iterations. 

Now let's talk about momentum. Let's say we have a cost function, with multiple local minimas. When trying to approach the global minimum, this can be an issue, because our model can essentially get stuck in a local minima. To fix this, we implement an idea called momentum, which is a technique that is used to nudge the model parameters from local minimas, Imagine a ball rolling down a hill. As it rolls, it gains momentum and goes faster and faster until it reaches the bottom. In the context of optimization, the "hill" represents the minimum, and the "ball" represents our model trying to find the minimum. The formula for momentum is given below

$M = M \times \beta - a \times \nabla(\theta)$

$\theta = \theta + M$

Here $\theta$ is our parameters, $a$ is the learning rate. $M$ is our momentum term for the parameters. Now we can apply both these concepts to update our optimizer class. 

In [10]:
class Optimizer_GD:
  def __init__(self, learning_rate=1e-3, decay=0., momentum=0.):
    self.learning_rate = learning_rate
    self.current_learning_rate = learning_rate
    self.decay = decay
    self.momentum = momentum
    self.iterations = 0
  def update_params(self, layer):
    if self.decay:
      self.current_learning_rate = self.learning_rate / (1 + self.decay * self.iterations)
    if self.momentum:
      if not hasattr(layer, 'weight_momentums'):
        layer.weight_momentums = np.zeros_like(layer.weights)
        layer.bias_momentums = np.zeros_like(layer.biases)
      weight_updates = layer.weight_momentums * self.momentum - self.current_learning_rate * layer.dweights
      layer.weight_momentums = weight_updates
      bias_updates = layer.bias_momentums * self.momentum - self.current_learning_rate * layer.dbiases
      layer.bias_momentums = bias_updates
    else:
      weight_updates = -self.current_learning_rate * layer.dweights
      bias_updates = -self.current_learning_rate * layer.dbiases
    layer.weights += weight_updates
    layer.biases += bias_updates
    self.iterations += 1

Now we have updated our Gradient Descent class. First, if the decay is a set parameter, we use the decay formula, to change the learning rate. Next, we initialize the momentums of our parameters, if they are not already there. Then using the momentum formula and gradients, we set our momentum, and use that to update our parameters. Now let's test it!

In [11]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

import pickle
with open('dataset.p', 'rb') as file:
  X, y = pickle.load(file)
X = X.reshape(X.shape[0], -1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

optimizer = Optimizer_GD(decay=1e-5, momentum=1e-3)

layer_one = Dense(784, 128)
relu_one = ReLu()
layer_two = Dense(128, 64)
relu_two = ReLu()
layer_three = Dense(64, 10)
softmax_loss = Softmax_Loss()

epochs = 10
batch_size = 32

for epoch in range(epochs):
  indices = np.arange(len(X_train))
  np.random.shuffle(indices)
  X_train, y_train = X_train[indices], y_train[indices]
  
  for i in range(0, len(X_train), batch_size):
    X_batch, y_batch = X_train[i:i+batch_size], y_train[i:i+batch_size]
     
    layer_one.forward(X_batch)
    relu_one.forward(layer_one.output)
    layer_two.forward(relu_one.output)
    relu_two.forward(layer_two.output)
    layer_three.forward(relu_two.output)
    loss = softmax_loss.forward(layer_three.output, y_batch)

    predictions = np.argmax(softmax_loss.output, axis=1)
    accuracy = np.mean(predictions == y_batch)

    softmax_loss.backprop(softmax_loss.output, y_batch)
    layer_three.backprop(softmax_loss.dinputs)
    relu_two.backprop(layer_three.dinputs)
    layer_two.backprop(relu_two.dinputs)
    relu_one.backprop(layer_two.dinputs)
    layer_one.backprop(relu_one.dinputs)

    optimizer.update_params(layer_one)
    optimizer.update_params(layer_two)
    optimizer.update_params(layer_three)

  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.5f}, Accuracy: {accuracy * 100:.2f}%')

layer_one.forward(X_test)
relu_one.forward(layer_one.output)
layer_two.forward(relu_one.output)
relu_two.forward(layer_two.output)
layer_three.forward(relu_two.output)

test_loss = softmax_loss.forward(layer_three.output, y_test)
print(f'Test Loss: {test_loss:.5f}') 

predictions = np.argmax(softmax_loss.output, axis=1) 
accuracy = np.mean(predictions == y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10, Loss: 0.53005, Accuracy: 84.38%
Epoch 2/10, Loss: 0.26834, Accuracy: 93.75%
Epoch 3/10, Loss: 0.26827, Accuracy: 96.88%
Epoch 4/10, Loss: 0.20454, Accuracy: 96.88%
Epoch 5/10, Loss: 0.23371, Accuracy: 96.88%
Epoch 6/10, Loss: 0.23881, Accuracy: 87.50%
Epoch 7/10, Loss: 0.08769, Accuracy: 96.88%
Epoch 8/10, Loss: 0.49848, Accuracy: 84.38%
Epoch 9/10, Loss: 0.22595, Accuracy: 90.62%
Epoch 10/10, Loss: 0.11739, Accuracy: 96.88%
Test Loss: 0.32194
Test Accuracy: 90.96%


Alright so it looks like our mode along with momentum and decay, did slightly increase our accuracy on the training and testing datasets, but let's look at a better optimizer.

## __AdaGrad and RMSProp__

AdaGrad is a type of optimizer, that instead of having a general learning rate for all parameters, has a per-parameter learning rate. During the training process, some weights can rise significantly more than others, and this can be a problem. To fix this, AdaGrad adapts learning rates of parameters by scaling them inverselty proportional to the square root of the sum of thier squared values, so parameters that recieve small updates, will have their learning rate increased, and vice versa. 

$G = G + g^2$

$\theta = \theta - \frac {LR}{\sqrt {G + e}} * g$

Here first the entire sum gets updated by the square of current gradient. Then we update the parameters, by inversely scaling to this value, and adding a hyperparameter called epsilon, to avoid division by 0. Now we can implement this.

In [12]:
class Optimizer_AdaGrad:
  def __init__(self, learning_rate=0.0001, epsilon=1e-7):
    self.learning_rate = learning_rate
    self.epsilon = epsilon
  def update_params(self, layer):
    if not hasattr(layer, 'weight_cache'):
      layer.weight_cache = np.zeros_like(layer.weights)
      layer.bias_cache = np.zeros_like(layer.biases)
    
    layer.weight_cache += layer.dweights ** 2
    layer.bias_cache += layer.dbiases ** 2

    layer.weights -= self.learning_rate * layer.dweights / np.sqrt(layer.weight_cache + self.epsilon)
    layer.biases -= self.learning_rate * layer.dbiases / np.sqrt(layer.bias_cache + self.epsilon)

Let's talk about problems with AdaGrad. The cache of the of sum of squared gradients, can get very large over time, causing learning rates to drop too much. This can slow down or halt learning during the training process. To fix this, we can introduce an optimizer called RMS_Prop, which introduces a moving average, which gives more weight to recent gradients, and helps in stopping learning rate from decreasing too fast. The only change is in the sum of the squared gradients, given below:

$G = \beta \times G + (1 - \beta) \times g^2$

$\theta = \theta - \frac {LR}{\sqrt {G + e}} * g$

Here, $\beta$ is a hyperparameter called rho. Now let's change the AdaGrad code a bit to make the RMSProp optimizer.

In [13]:
class Optimizer_RMSProp:
  def __init__(self, learning_rate=0.0001, epsilon=1e-7, rho=0.9):
    self.learning_rate = learning_rate
    self.epsilon = epsilon
    self.rho = rho
  def update_params(self, layer):
    if not hasattr(layer, 'weight_cache'):
      layer.weight_cache = np.zeros_like(layer.weights)
      layer.bias_cache = np.zeros_like(layer.biases)
    
    layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights ** 2
    layer.bias_cache = self.rho * layer.bias_cache + (1 - self.rho) * layer.dbiases ** 2

    layer.weights -= self.learning_rate * layer.dweights / np.sqrt(layer.weight_cache + self.epsilon)
    layer.biases -= self.learning_rate * layer.dbiases / np.sqrt(layer.bias_cache + self.epsilon)

Now let's test our optimizer

In [14]:
optimizer = Optimizer_RMSProp()

for epoch in range(epochs):
  indices = np.arange(len(X_train))
  np.random.shuffle(indices)
  X_train, y_train = X_train[indices], y_train[indices]
  
  for i in range(0, len(X_train), batch_size):
    X_batch, y_batch = X_train[i:i+batch_size], y_train[i:i+batch_size]
     
    layer_one.forward(X_batch)
    relu_one.forward(layer_one.output)
    layer_two.forward(relu_one.output)
    relu_two.forward(layer_two.output)
    layer_three.forward(relu_two.output)
    loss = softmax_loss.forward(layer_three.output, y_batch)

    predictions = np.argmax(softmax_loss.output, axis=1)
    accuracy = np.mean(predictions == y_batch)

    softmax_loss.backprop(softmax_loss.output, y_batch)
    layer_three.backprop(softmax_loss.dinputs)
    relu_two.backprop(layer_three.dinputs)
    layer_two.backprop(relu_two.dinputs)
    relu_one.backprop(layer_two.dinputs)
    layer_one.backprop(relu_one.dinputs)

    optimizer.update_params(layer_one)
    optimizer.update_params(layer_two)
    optimizer.update_params(layer_three)

  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.5f}, Accuracy: {accuracy * 100:.2f}%')

layer_one.forward(X_test)
relu_one.forward(layer_one.output)
layer_two.forward(relu_one.output)
relu_two.forward(layer_two.output)
layer_three.forward(relu_two.output)

test_loss = softmax_loss.forward(layer_three.output, y_test)
print(f'Test Loss: {test_loss:.5f}') 

predictions = np.argmax(softmax_loss.output, axis=1) 
accuracy = np.mean(predictions == y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10, Loss: 0.01153, Accuracy: 100.00%
Epoch 2/10, Loss: 0.17010, Accuracy: 93.75%
Epoch 3/10, Loss: 0.08582, Accuracy: 96.88%
Epoch 4/10, Loss: 0.22914, Accuracy: 93.75%
Epoch 5/10, Loss: 0.37356, Accuracy: 90.62%
Epoch 6/10, Loss: 0.23639, Accuracy: 96.88%
Epoch 7/10, Loss: 0.02806, Accuracy: 100.00%
Epoch 8/10, Loss: 0.06333, Accuracy: 96.88%
Epoch 9/10, Loss: 0.61633, Accuracy: 93.75%
Epoch 10/10, Loss: 0.05929, Accuracy: 96.88%
Test Loss: 0.25189
Test Accuracy: 94.93%


Nice, it looks like we had an improvment in our training results! But now, we will be looking at the most widely used optimizer, Adam. It combines the ideas of RMSProp and Momentum, and adds correction for the initializing setting the momentums and caches to 0. 

## __Adam Optimizer__

The adam optimizer combines features of RMSProp and Momentum, and is the most popular optimizer due to it's efficiency across different types of problems. Let' look at the mathematics behind Adam.

Momentum
$M = \beta_{1} \times M + (1 - \beta_{1}) \times g$

Cache
$G = \beta_{2} \times G + (1 - \beta_{2}) \times g^2$

Corrected Momentum
#### $\hat M = \frac {M}{1 - \beta_{1}^t}$

Correct Cache
#### $\hat G = \frac {G}{1 - \beta_{2}^t}$

Update values
#### $\theta = \theta - \frac {LR}{\sqrt {\hat G} + e} \times \hat M$

Now let's implement it!

In [15]:
class Optimizer_Adam:
  def __init__(self, epsilon=1e-7, learning_rate=0.001, beta_1=0.9, beta_2=0.999):
    self.learning_rate = learning_rate
    self.epsilon = epsilon
    self.beta_1 = beta_1
    self.beta_2 = beta_2
    self.iterations = 0
  def update_params(self, layer):
    if not hasattr(layer, 'weight_cache'):
      layer.weight_cache = np.zeros_like(layer.weights)
      layer.bias_cache = np.zeros_like(layer.biases)
      layer.weight_momentums = np.zeros_like(layer.weights)
      layer.bias_momentums = np.zeros_like(layer.biases)
    layer.weight_momentums = self.beta_1 * layer.weight_momentums  + (1 - self.beta_1) * layer.dweights
    layer.bias_momentums = self.beta_2 * layer.bias_momentums  + (1 - self.beta_2) * layer.dbiases
    weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
    bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_2 ** (self.iterations + 1))
    layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights ** 2
    layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases ** 2
    weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
    bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))
    layer.weights -= self.learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
    layer.biases -= self.learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

Nice, now we have finished implementing the Adam optimizer, let's test it out.

In [16]:
optimizer = Optimizer_Adam()

layer_one = Dense(784, 128)
relu_one = ReLu()
layer_two = Dense(128, 64)
relu_two = ReLu()
layer_three = Dense(64, 10)
softmax_loss = Softmax_Loss()

for epoch in range(epochs):
  indices = np.arange(len(X_train))
  np.random.shuffle(indices)
  X_train, y_train = X_train[indices], y_train[indices]
  
  for i in range(0, len(X_train), batch_size):
    X_batch, y_batch = X_train[i:i+batch_size], y_train[i:i+batch_size]
     
    layer_one.forward(X_batch)
    relu_one.forward(layer_one.output)
    layer_two.forward(relu_one.output)
    relu_two.forward(layer_two.output)
    layer_three.forward(relu_two.output)
    loss = softmax_loss.forward(layer_three.output, y_batch)

    predictions = np.argmax(softmax_loss.output, axis=1)
    accuracy = np.mean(predictions == y_batch)

    softmax_loss.backprop(softmax_loss.output, y_batch)
    layer_three.backprop(softmax_loss.dinputs)
    relu_two.backprop(layer_three.dinputs)
    layer_two.backprop(relu_two.dinputs)
    relu_one.backprop(layer_two.dinputs)
    layer_one.backprop(relu_one.dinputs)

    optimizer.update_params(layer_one)
    optimizer.update_params(layer_two)
    optimizer.update_params(layer_three)

  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.5f}, Accuracy: {accuracy * 100:.2f}%')

layer_one.forward(X_test)
relu_one.forward(layer_one.output)
layer_two.forward(relu_one.output)
relu_two.forward(layer_two.output)
layer_three.forward(relu_two.output)

test_loss = softmax_loss.forward(layer_three.output, y_test)
print(f'Test Loss: {test_loss:.5f}') 

predictions = np.argmax(softmax_loss.output, axis=1) 
accuracy = np.mean(predictions == y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10, Loss: 0.81498, Accuracy: 71.88%
Epoch 2/10, Loss: 0.63102, Accuracy: 84.38%
Epoch 3/10, Loss: 0.13327, Accuracy: 90.62%
Epoch 4/10, Loss: 0.01897, Accuracy: 100.00%
Epoch 5/10, Loss: 0.07676, Accuracy: 96.88%
Epoch 6/10, Loss: 0.14442, Accuracy: 96.88%
Epoch 7/10, Loss: 0.02162, Accuracy: 100.00%
Epoch 8/10, Loss: 0.16082, Accuracy: 96.88%
Epoch 9/10, Loss: 0.09315, Accuracy: 96.88%
Epoch 10/10, Loss: 0.00638, Accuracy: 100.00%
Test Loss: 0.16481
Test Accuracy: 95.86%


There we go! Our model has a near perfect accuracy on the training dataset, and a very high accuracy on the testing dataset. That's it for this project. There are two other notebooks, one with the full code, and one with all the math formulas important for this project. Also feel free, to clone this repository, and edit things according to your liking, such as adding functions to improve efficiency, or expanding with more optimizers and loss functions, and even try it on other datasets!